In [1]:
import dash
import pathlib
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import plotly.express as px
from dash.dependencies import Input, Output
import numpy as np
import pandas as pd
import math

In [2]:
colors = {'plot_color':'#B31232', 'paper_color':'#000000'}

In [3]:
def runge_Kutt(f, g, t0, x0, y0, h, b):
    t = t0
    x = x0
    y = y0
    arrayX = np.array([])
    arrayY = np.array([])
    arrayT = np.array([])
    while t<b:
        k1 = g(t, x, y)
        q1 = f(t, x, y)
        
        k2 = g(t + h/2, x + (h*q1)/2, y + (h*k1)/2)
        q2 = f(t + h/2, x + (h*q1)/2, y + (h*k1)/2)
        
        k3 = g(t + h/2, x + (h*q2)/2, y + (h*k2)/2)
        q3 = f(t + h/2, x + (h*q2)/2, y + (h*k2)/2)
        
        k4 = g(t + h, x + h*q3, y + h*k3)
        q4 = f(t + h, x + h*q3, y + h*k3)
        
        y = y + h*(k1 + 2*k2 + 2*k3 + k4)/6
        x = x + h*(q1 + 2*q2 + 2*q3 + q4)/6
        
        arrayX = np.append(arrayX, x)
        arrayT = np.append(arrayT, t)
        arrayY = np.append(arrayY, y)
        
        t += h
    return arrayT, arrayX, arrayY

## Creating Dashboard

In [29]:
app = dash.Dash(__name__)

app.layout = html.Div([
    #header
    html.Div([
        html.H1(children='Two competing species',
           style={'textAlign':'center','color':'#FFFFFF'})
             ], className="app__header"),
    #control center
    html.Div([
    html.Div([
        html.Div([html.Div(id='my-output1', children=[], className='graph__title'),
                  dcc.Slider(id='slider1', min=0, max=3, step=0.01, value=1, marks={i: i for i in range(4)})]),
        html.Div([html.Div(id='my-output2', children=[], className='graph__title'),
                  dcc.Slider(id='slider2', min=0, max=3, step=0.01, value=1, marks={i: i for i in range(4)})]),
        html.Div([html.Div(id='my-output3', children=[], className='graph__title'),
                  dcc.Slider(id='slider3', min=0, max=3, step=0.01, value=0.2, marks={i: i for i in range(4)})]),
        html.Div([html.Div(id='my-output4', children=[], className='graph__title'),
                  dcc.Slider(id='slider4', min=0, max=3, step=0.01, value=0.3, marks={i: i for i in range(4)})]),
        html.Div([html.Div(id='my-output5', children=[], className='graph__title'),
                  dcc.Slider(id='slider5', min=0, max=3, step=0.01, value=0.205, marks={i: i for i in range(4)})]),
        html.Div([html.Div(id='my-output6', children=[], className='graph__title'),
                  dcc.Slider(id='slider6', min=0, max=3, step=0.01, value=0.25, marks={i: i for i in range(4)})]),
        
        
    ], className='control__center'),
    #graphics    
    html.Div([
        html.Div([html.H6("Interaction of species", className="graph__title")]),
        
        dcc.Graph(id='plot', figure={})], className='graph__container')
    ], className = 'app__content')

], className="app__container")


@app.callback(
    [Output(component_id='my-output1', component_property='children'),
     Output(component_id='my-output2', component_property='children'),
     Output(component_id='my-output3', component_property='children'),
     Output(component_id='my-output4', component_property='children'),
     Output(component_id='my-output5', component_property='children'),
     Output(component_id='my-output6', component_property='children'),
    Output(component_id='plot', component_property='figure')],
    [Input(component_id='slider1', component_property='value'),
    Input(component_id='slider2', component_property='value'),
    Input(component_id='slider3', component_property='value'),
    Input(component_id='slider4', component_property='value'),
    Input(component_id='slider5', component_property='value'),
    Input(component_id='slider6', component_property='value')]
)
def update_output_div(a1, a2, b11, b12, b21, b22):
    o1, o2, o3, o4, o5, o6 = f'a1: {a1}',f'a2: {a2}', f'b11: {b11}',  f'b12: {b12}',  f'b21: {b21}',  f'b22: {b22}'
    
    listx = np.array([])
    listy = np.array([])
    
#     a1 = a1
#     a2 = a2
#     b11 = b11
#     b12 = b12
#     b21 = 0
#     b22 = 0.25
    for i in range(5):
        p1, p2, p3 = runge_Kutt(lambda t,x,y: a1*x - b11*x**2 - b12*x*y, 
                                lambda t,x,y: a2*y - b21*x*y - b22*y**2, 0,
                                math.cos(i*np.pi/60), math.sin(i*np.pi/60), 0.01, 100)
        listx = np.concatenate((listx, p2))
        listy = np.concatenate((listy, p3))
        
        list1 = np.array([listx, listy]).T
        df = pd.DataFrame(list1, columns=['x', 'y'])    

        df1 = pd.DataFrame(np.array([[0, a1/b11, 0, (b12*a2-b22*a1)/(b21*b12-b22*b11)],\
                                     [0, 0, a2/b22,(b21*a1-b11*a2)/(b21*b12-b22*b11)]]).T, columns=['x', 'y'])
        fig = go.Figure(data=go.Scatter(x=df['x'],y=df['y'], mode='lines'))
#         fig = px.scatter(df, x='x', y='y')
        
        fig.add_trace(go.Scatter(x=df1['x'], y=df1['y'], mode='markers'))
        fig.update_layout(transition_duration=500)
    
    return o1, o2, o3, o4, o5, o6, fig


if __name__ == '__main__':
    app.run_server(port=8001)

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is running on http://127.0.0.1:8001/

Dash is run

 * Running on http://127.0.0.1:8001/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Nov/2020 21:25:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Nov/2020 21:25:08] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [02/Nov/2020 21:25:08] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [02/Nov/2020 21:25:10] "POST /_dash-update-component HTTP/1.1" 200 -
